# change the API_KEY to the key for guardian  
# change the OPENAIKEY to the key for openai

In [ ]:
from capture import get_content
from capture import fetch_article_content
from capture import event_type_classification
from capture import get_title_and_content
from capture import filter_news
from capture import get_casualty_dict
# Replace 'YOUR_API_KEY' with your actual API key from The Guardian
API_KEY = ''

import os
os.environ["OPENAI_API_KEY"] =""

from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

from tqdm import tqdm
from json import JSONDecodeError


# filter_news, param:

# startdate, enddate, location, apikey

In [ ]:
result = filter_news("2023-1-20","2023-9-27", "Gaza", API_KEY)
# result

In [ ]:
casualty_dict_list = []

for date in tqdm(result):

    for article in result[date]:
        main_content = article['main_content']

        try:
            casualty_dict = get_casualty_dict(main_content)
            
            # only append the casualty_dict if it is not None and the number of victims is greater than 0
            if casualty_dict is not None and casualty_dict['event_number_of_victims'] > 0:
                # add the date and the url to the casualty_dict
                casualty_dict['date'] = date
                casualty_dict['url'] = article['url']
                casualty_dict['title'] = article['title']
                
                casualty_dict_list.append(casualty_dict)

            print(casualty_dict)
        
        except JSONDecodeError:
            print(f"Error decoding JSON for article URL: {article['url']}")
        except Exception as e:  # This is a general exception to catch any other unexpected errors.
            print(f"An error occurred for article URL: {article['url']}. Error message: {str(e)}")


In [ ]:
print(casualty_dict_list)

# save into the json file

In [ ]:
# save the casualty_dict_list to a json file, in a readable format
import json
with open('casualty_dict_list.json', 'w') as outfile:
    json.dump(casualty_dict_list, outfile, indent=4)